### Vorbereitung der Programmierumgebung

In [15]:
import os
import numpy as np
from arcpy.sa import *

In [3]:
# Umgebungseinstellungen
arcpy.env.workspace = r"Beispieldaten"  # Pfad zum Arbeitsverzeichnis
arcpy.env.overwriteOutput = True
os.chdir(arcpy.env.workspace)

### Daten einladen

In [4]:
# Verzeichnis mit den Eingabe-Rastern (SI-Raster)
si_folder = r"data/Statistical_Index"
raster_list = [f for f in os.listdir(si_folder) if f.endswith("SI.tif")]

# Ordner, in dem deine .dbf Tabellen liegen
tabulate_folder = r"data/Statistical_Index"
# Liste aller .dbf-Dateien, die mit "Tabulate_" beginnen
table_list = [f for f in os.listdir(tabulate_folder) if f.startswith("Tabulate_") and f.endswith(".dbf")]

# Ausgabeordner
output_folder = r"data/Weighting_Faktor"
# Ordner anlegen, wenn er nicht existiert
os.makedirs(output_folder, exist_ok=True)


### Berechnung des TS für jeden Parameter

In [7]:
results = []  # Für spätere Gewichtung

for raster_name in raster_list:
    param_name = raster_name.replace("_SI.tif", "")
    si_path = os.path.join(si_folder, raster_name)
    
    # Passende Tabelle finden
    matching_table = f"Tabulate_{param_name}.dbf"
    table_path = os.path.join(tabulate_folder, matching_table)

    if not os.path.exists(table_path):
        print(f"⚠️ Keine Tabelle für {param_name} gefunden – wird übersprungen.")
        continue

    print(f"🔄 Bearbeite Parameter: {param_name}")

    # Tabelle laden: Value = Klassenwert, SUM = Anzahl Rutschungspixel in der Klasse
    zonal_table = arcpy.da.TableToNumPyArray(table_path, ["Value", "SUM"])

    # SI-Raster laden als Dictionary (Klasse: SI-Wert)
    si_raster = arcpy.Raster(si_path)
    si_array = arcpy.RasterToNumPyArray(si_raster)
    unique_classes = np.unique(si_array)

    # Dictionary: Klasse → SI-Wert (angenommen, der SI-Wert ist direkt der Zellwert)
    si_dict = {val: val for val in unique_classes if val != si_raster.noDataValue}

    # Total Score (TS_i) berechnen
    ts = 0.0
    for row in zonal_table:
        cls = row[0]
        n_pix = row[1]
        si = si_dict.get(cls, 0)  # Fallback auf 0, falls Klasse im SI-Raster fehlt
        ts += n_pix * si

    results.append((param_name, ts))
    print(f"✅ {param_name}: TS = {ts:.4f}")

# --- 3. Optional: In Textdatei speichern ---
output_txt = os.path.join(output_folder, "Total_Scores.txt")
with open(output_txt, "w") as f:
    for name, ts in results:
        f.write(f"{name}\t{ts:.6f}\n")

print(f"\n📁 Alle TS_i-Werte gespeichert unter:\n{output_txt}")

🔄 Bearbeite Parameter: aspect_reclass_reproject
✅ aspect_reclass_reproject: TS = 6184.0000
🔄 Bearbeite Parameter: curvature_planar_reclass_reproject
✅ curvature_planar_reclass_reproject: TS = 2903.0000
🔄 Bearbeite Parameter: curvature_profile_reclass_reproject
✅ curvature_profile_reclass_reproject: TS = 3113.0000
🔄 Bearbeite Parameter: litho1_reprojected
✅ litho1_reprojected: TS = 4429.0000
🔄 Bearbeite Parameter: precip_va3dayinterval_ordkriging_AOI_reclass_reproject
✅ precip_va3dayinterval_ordkriging_AOI_reclass_reproject: TS = 3495.0000
🔄 Bearbeite Parameter: Reclass_SPI_reproject
✅ Reclass_SPI_reproject: TS = 1725.0000
🔄 Bearbeite Parameter: rivers_setnull_mrb500_100_viewshed_reproject
✅ rivers_setnull_mrb500_100_viewshed_reproject: TS = 6486.0000
🔄 Bearbeite Parameter: roads_mrb250_50_viewshed_reclass_reproject
✅ roads_mrb250_50_viewshed_reclass_reproject: TS = 2788.0000
🔄 Bearbeite Parameter: slope_reclass_reprojected
✅ slope_reclass_reprojected: TS = 3893.0000
🔄 Bearbeite Paramet

### Berechnung Weighting Faktor (normiert von 0 bis 1)

In [11]:
# Extrahieren der Total Scores aus den Ergebnissen
total_scores = [score for _, score in results]

# Minimum und Maximum der Total Scores
min_ts = min(total_scores)
max_ts = max(total_scores)

# Normierte Gewichtungsfaktoren (Skala 0–1)
weighting_factors = [(ts - min_ts) / (max_ts - min_ts)*9+1 for ts in total_scores]

In [12]:
print(weighting_factors)
print(results)

[9.429111531190927, 3.226843100189036, 3.623818525519849, 6.111531190926276, 4.34593572778828, 1.0, 10.0, 3.0094517958412097, 5.098298676748582, 6.886578449905482, 1.4669187145557656, 5.508506616257089]
[('aspect_reclass_reproject', 6184.0), ('curvature_planar_reclass_reproject', 2903.0), ('curvature_profile_reclass_reproject', 3113.0), ('litho1_reprojected', 4429.0), ('precip_va3dayinterval_ordkriging_AOI_reclass_reproject', 3495.0), ('Reclass_SPI_reproject', 1725.0), ('rivers_setnull_mrb500_100_viewshed_reproject', 6486.0), ('roads_mrb250_50_viewshed_reclass_reproject', 2788.0), ('slope_reclass_reprojected', 3893.0), ('soil1_repro', 4839.0), ('twi_reclass_reprojected', 1972.0), ('VGTB_LandCover_2010_v2_reprojected', 4110.0)]


In [16]:
# Liste der vollständigen SI-Rasterpfade
si_raster_paths = [os.path.join(si_folder, r) for r in raster_list] # wurde oben definiert

# Lookup-Raster erstellen (temporär, da SI-Werte nicht direkt addiert werden können)
lookup_rasters = [Lookup(raster, "SI") for raster in si_raster_paths]

# Gewichts-Raster erstellen: Jedes SI-Raster mit seinem jeweiligen Gewichtungsfaktor multiplizieren
weighted_rasters = []
for idx, raster in enumerate(lookup_rasters):
    weighted_raster = raster * weighting_factors[idx]  # SI-Raster mit Gewichtungsfaktor multiplizieren
    weighted_rasters.append(weighted_raster)

# Summe der gewichteten Raster berechnen
si_sum = CellStatistics(weighted_rasters, "SUM", "DATA")

# Ergebnis speichern
si_sum.save(os.path.join(output_folder, "SI_sum_weighted.tif"))  # Summe der gewichteten Raster speichern
print("Gefährdungslayer mit Gewichtungsfaktoren wurde erfolgreich erstellt.")

Gefährdungslayer mit Gewichtungsfaktoren wurde erfolgreich erstellt.


In [ ]:
### SI-Weighted normieren (0-1)

In [18]:
# Min und Max Werte des si_sum Layers berechnen
min_value = arcpy.management.GetRasterProperties(si_sum, "MINIMUM")
max_value = arcpy.management.GetRasterProperties(si_sum, "MAXIMUM")

# Min und Max als Variablen extrahieren
min_value = float(min_value.getOutput(0))
max_value = float(max_value.getOutput(0))

# Raster standardisieren: (Wert - Min) / (Max - Min)
standardized_raster = (Raster(si_sum) - min_value) / (max_value - min_value)

# Ausgabe-Raster speichern
standardized_raster.save(os.path.join(output_folder, "SI_sum_weighted_standardized.tif"))
